# Data Exploration - Setup - Splitting

## Imports

In [7]:
# Data Handling
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Visualization
import matplotlib.pyplot as plt

# Paths
from src.__00__paths import curated_data_dir, raw_data_dir, processed_data_dir
import shutil
from pathlib import Path

# Datasets Source
import kagglehub

## Download Walmart Sales Forecast Data (Kaggle)

In [11]:
# Data files to check
data_items = [
    raw_data_dir / "features.csv",
    raw_data_dir / "stores.csv",
    raw_data_dir / "test.csv",
    raw_data_dir / "train.csv",
]

# Check and download
if all(item.exists() for item in data_items):
    print("Dataset are already downloaded.")
else:
    print("Downloading data...")
    dataset_path = Path(kagglehub.dataset_download("aslanahmedov/walmart-sales-forecast"))

    if not dataset_path.exists():
        raise FileNotFoundError("Dataset not found.")

    # Copy all files from KaggleHub Cache to `data/raw`
    for item in dataset_path.iterdir():
        target = raw_data_dir / item.name
        shutil.copy2(item, target)
    print("Dataset successfully downloaded.")

Dataset are already downloaded.
